In [2]:
import pandas as pd

def replace_first_space(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            first_space_index = line.find(' ')
            modified_line = line[:first_space_index] + ';' + line[first_space_index + 1:]
            outfile.write(modified_line)

In [3]:
replace_first_space("./original_outputs/bad0.smi", "./parsed_outputs/parsed_smi_0.smi")
replace_first_space("./original_outputs/bad1.smi", "./parsed_outputs/parsed_smi_1.smi")
replace_first_space("./original_outputs/bad2.smi", "./parsed_outputs/parsed_smi_2.smi")
replace_first_space("./original_outputs/bad3.smi", "./parsed_outputs/parsed_smi_3.smi")

In [8]:
df0 = pd.read_csv("./parsed_outputs/parsed_smi_0.smi",
                  delimiter=";", names=["SMILES", "FlagReason"])
df1 = pd.read_csv("./parsed_outputs/parsed_smi_1.smi",
                  delimiter=";", names=["SMILES", "FlagReason"])
df2 = pd.read_csv("./parsed_outputs/parsed_smi_2.smi",
                  delimiter=";", names=["SMILES", "FlagReason"])
df3 = pd.read_csv("./parsed_outputs/parsed_smi_3.smi",
                  delimiter=";", names=["SMILES", "FlagReason"])
df = pd.concat([df0, df1, df2, df3])
df

,SMILES,FlagReason
0,C=CCN=C=S,TP1 not_enough_atoms
1,C1=CC(Cl)=C2C(=CC=CC2=C1)C1=NC=C2C(=NC(=NC2=C1...,TP1 too_many_atoms
2,C(Cl)C1OC1,TP1 not_enough_atoms
3,C(Br)C1OC1,TP1 not_enough_atoms
4,C(F)C1OC1,TP1 not_enough_atoms
...,...,...
33,C1=NN2C=CC(=CC2=C1C1=CC=C(C(=O)NC)C=C1)C(=O)N(...,: D(105) too_many_atoms:ester:aniline_acyl_ewd
34,C1=CC(=CC=C1F)C(C1=CC=C(F)C=C1)N1CCN(C2=C(N(=O...,: D(146) too_many_atoms:nitro
35,OC[C@H]1O[C@@H](O[C@@H]2OC=C3C(=O)OCC[C@@]3(O)...,: D(116) acetal_1_in_ring:michael_demerited:to...
36,C1=CC(=CC2=C1OCCO2)C1=NN(C(=O)C)[C@@H](C2=CC=C...,: D(113) nitro:acyl_hydrazone_cyclic:too_many_...


In [12]:
df_fail = df[~df.FlagReason.str.contains("match")]
df_match = df[df.FlagReason.str.contains("match")]